In [1]:
import pandas as pd
from geopandas import GeoDataFrame, read_file

import sys
sys.path.append("..")
import movingpandas as mpd
mpd.show_versions()

import warnings
warnings.simplefilter("ignore")

/home/arg/miniconda3/envs/movingpandas/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)



MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.9.19 | packaged by conda-forge | (main, Mar 20 2024, 12:50:21)  [GCC 12.3.0]
executable : /home/arg/miniconda3/envs/movingpandas/bin/python
machine    : Linux-5.15.0-102-generic-x86_64-with-glibc2.31

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.8.4
GDAL data dir: /home/arg/miniconda3/envs/movingpandas/share/gdal
PROJ       : 9.3.1
PROJ data dir: /home/arg/miniconda3/envs/movingpandas/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.2.1
fiona      : 1.9.6
numpy      : 1.26.4
shapely    : 2.0.3
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.8.3
mapclassify: 2.6.1
geopy      : 2.4.1
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : 1.2


In [2]:
import csv
import os
import shutil
import xml.etree.ElementTree as ET
from datetime import timedelta

import numpy as np
from dateutil import parser


def generate_grid_and_path(x_min, x_max, x_n, y_min, y_max, y_n):
    x_values = np.linspace(x_min, x_max, x_n)
    y_values = np.linspace(y_min, y_max, y_n)
    X, Y = np.meshgrid(x_values, y_values)
    path = []

    for i in range(x_n):
        for j in range(y_n):
            if i % 2 == 0:
                path.append((X[j, i], Y[j, i]))
            else:
                path.append((X[-j - 1, i], Y[-j - 1, i]))

    if x_n % 2 == 0:
        for j in range(y_n):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    else:
        for j in range(y_n - 1, -1, -1):
            for i in range(x_n):
                if j % 2 == 0:
                    path.append((X[j, i], Y[j, i]))
                else:
                    path.append((X[j, -i - 1], Y[j, -i - 1]))
    return path


def parse_corrdinates_str_list(kml_path):
    tree = ET.parse(kml_path)
    root = tree.getroot()
    namespace = {"kml": "http://www.opengis.net/kml/2.2"}
    specific_name = "vehicle_global_position:14"
    coordinates = None
    coordinates_str_list = []
    for placemark in root.findall(".//kml:Placemark", namespaces=namespace):
        name = placemark.find("kml:name", namespaces=namespace)
        if name is not None and name.text == specific_name:
            # Find the LineString element
            line_string = placemark.find(".//kml:LineString", namespaces=namespace)
            if line_string is not None:
                # Extract the coordinates
                coordinates = line_string.find("kml:coordinates", namespaces=namespace)
                if coordinates is not None:
                    coordinates_str_list.append(coordinates.text.strip())
    return coordinates_str_list


def get_means(coordinates):
    mean_x = np.mean([coord[0] for coord in coordinates])
    mean_y = np.mean([coord[1] for coord in coordinates])
    return mean_x, mean_y


def write_to_csv(
    csv_file_path, coordinates, trajectory_id, tracker, base_timestamp, mean_x, mean_y, grid_size_half=10e-5, xy_n=10
):
    base_time = parser.parse(base_timestamp)
    with open(csv_file_path, "w", newline="") as csv_file:
        writer = csv.writer(csv_file, delimiter=";")

        headers = ["X", "Y", "fid", "id", "sequence", "trajectory_id", "tracker", "t"]
        writer.writerow(headers)
        for i, coord in enumerate(coordinates, start=1):
            x, y, _ = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id, tracker, timestamp]
            writer.writerow(row)

        path = generate_grid_and_path(
            mean_x - grid_size_half,
            mean_x + grid_size_half,
            xy_n,
            mean_y - grid_size_half,
            mean_y + grid_size_half,
            xy_n,
        )
        for i, coord in enumerate(path, start=len(coordinates) + 1):
            x, y = coord
            timestamp = (base_time + timedelta(seconds=1 * (i - 1))).isoformat()
            row = [x, y, i, i, i, trajectory_id + 1, tracker, timestamp]
            writer.writerow(row)

    print(f"Saved to {csv_file_path}")


def convert_kml_to_csv(kml_path):
    kml_name = kml_path.split("/")[-1].split(".")[0]

    coordinates_str_list = parse_corrdinates_str_list(kml_path)[0].split()
    coordinates = [[float(coord) for coord in coord_str.split(",")] for coord_str in coordinates_str_list]
    mean_x, mean_y = get_means(coordinates)
    print(f"Min x: {min([coord[0] for coord in coordinates])}, Min y: {min([coord[1] for coord in coordinates])}")
    print(f"Max x: {max([coord[0] for coord in coordinates])}, Max y: {max([coord[1] for coord in coordinates])}")
    xy_n = 11
    grid_size_half = 1 / 111_111 * xy_n # Convert to meter

    print(f"Mean x: {mean_x}, Mean y: {mean_y}")

    trajectory_id = 1
    tracker = 19
    base_timestamp = "2008-12-11 04:42:14+00"

    csv_file_path = f"./data/{kml_name}.csv"
    write_to_csv(
        csv_file_path,
        coordinates,
        trajectory_id,
        tracker,
        base_timestamp,
        mean_x,
        mean_y,
        grid_size_half,
        xy_n=xy_n,
    )
    # shutil.copy(csv_file_path, os.path.expanduser("./data/"))
    print(f"Copied to ./data/{kml_name}.csv")


kml_pathes = [

    "log_63_2024-3-5-16-37-26.kml",
    # "log_54_2024-3-5-15-24-1.kml",
    # "log_50_2024-3-5-14-39-40.kml",
    # "log_51_2024-3-5-14-45-46.kml",
    # "log_57_2024-3-5-16-08-38.kml"
    # "log_59_2024-3-5-16-15-24.kml"
]


kml_pathes = [f"./data/{kml_path}" for kml_path in kml_pathes]

for kml_path in kml_pathes:
    convert_kml_to_csv(kml_path)

Min x: 120.99334692788509, Min y: 24.78697099516003
Max x: 120.99337998269704, Max y: 24.787024335003476
Mean x: 120.99336349961983, Mean y: 24.787003676108693
Saved to ./data/log_63_2024-3-5-16-37-26.csv
Copied to ./data/log_63_2024-3-5-16-37-26.csv


In [3]:
import geoviews as gv
bing_maps_tile_source = gv.tile_sources.WMTS("http://ecn.t3.tiles.virtualearth.net/tiles/a{q}.jpeg?g=1")
hvplot_defaults = {'tiles':bing_maps_tile_source, 'frame_height':700, 'frame_width':700, 'colorbar':True}

In [4]:
%%time 
df = pd.read_csv('data/log_63_2024-3-5-16-37-26.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 576 ms, sys: 40 ms, total: 616 ms
Wall time: 617 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/log_53_2024-3-5-15-16-12.png)

In [6]:
%%time 
df = pd.read_csv('data/log_53_2024-3-5-15-16-12.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

CPU times: user 148 ms, sys: 421 µs, total: 148 ms
Wall time: 148 ms


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Path.I    :Path   [Longitude,Latitude]
   .Path.II   :Path   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II :Points   [Longitude,Latitude]   (triangle_angle)

![alt text](img/log_54_2024-3-5-15-24-18.png)

In [ ]:
%%time 
df = pd.read_csv('data/log_50_2024-3-5-14-39-40.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], **hvplot_defaults)

![alt text](img/log_50_2024-3-5-14-39-40.png)

In [ ]:
%%time 
df = pd.read_csv('data/log_51_2024-3-5-14-45-46.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], color=["orange", "red"], **hvplot_defaults)

![alt text](img/log_51_2024-3-5-14-45-46.png)

In [ ]:
%%time 
df = pd.read_csv('data/log_57_2024-3-5-16-08-38.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], color=["orange", "red"], **hvplot_defaults)

![alt text](img/log_57_2024-3-5-16-08-38.png)

In [ ]:
%%time 
df = pd.read_csv('data/log_59_2024-3-5-16-15-24.csv', delimiter=';')
traj_collection = mpd.TrajectoryCollection(df, 'trajectory_id', t='t', x='X', y='Y')
traj_collection.hvplot(title=str(traj_collection), line_width=[5,.8], color=["orange", "red"], **hvplot_defaults)

In [ ]:
import pandas as pd


![alt text](img/log_59_2024-3-5-16-15-24.png)